In [95]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras import callbacks, optimizers
import pdf2image
import pickle

In [96]:
path = '/Users/skylark/Desktop/pdf_to_img'
os.listdir(path)

['untitled folder', '.DS_Store', 'acceptable', 'not_acceptable']

In [97]:
folders = ['acceptable', 'not_acceptable']

In [98]:
folders = ['acceptable', 'not_acceptable']

acceptable = []
not_acceptable = []

for folder in folders:
    path_new = path + '/' + folder
    if folder == 'acceptable':
        for img in os.listdir(path_new):
            if img == '.DS_Store':
                pass
            else:
                pic = image.load_img(path_new+'/'+img, target_size=(300, 300))
                acceptable.append(image.img_to_array(pic))
                print(path_new+'/'+img)
    elif folder == 'not_acceptable':
        for img in os.listdir(path_new):
            if img == '.DS_Store':
                pass
            else:
                pic = image.load_img(path_new+'/'+img, target_size=(300,300))
                not_acceptable.append(image.img_to_array(pic))
                print(path_new+'/'+img)

/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_327.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_333.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_482.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_125.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output1python_279.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output1aws_99.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output1python_319.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_14.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output1aws_163.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_247.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_5.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output1aws_2_20.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_4.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_252.jpg
/Users/skylark/Desktop/pdf_to_img/acceptable/output0new_15.jpg
/Users/skylark/Desktop/pdf_to_img/accept

In [99]:
len(acceptable), len(not_acceptable[:1100])

(403, 475)

In [100]:
len(acceptable) + len(not_acceptable[:1100])

878

In [101]:
acceptable = np.array(acceptable)/255
not_acceptable = np.array(not_acceptable)/255
images = np.concatenate((acceptable, not_acceptable), axis=0)

target = np.ones(shape=(len(acceptable) + len(not_acceptable),))
target[403:] = 0

In [102]:
len(images), len(target)

(878, 878)

In [104]:
from keras.applications import InceptionV3, Xception

In [105]:
inception = Xception(include_top=False, input_shape=(300,300,3))

In [106]:
inception.trainable = False

set_trainable = False
for layer in inception.layers:
    if layer.name == 'conv2d_3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True        

In [107]:
x = keras.layers.Flatten()(inception.output)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.models.Model(inception.input, x)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [111]:
train_x, test_x, train_y, test_y = train_test_split(images, target, test_size = 0.1, random_state=48)

In [114]:
call_backs = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(factor=0.01)]

In [115]:
model.compile(optimizer=optimizers.RMSprop(0.005), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(train_x, train_y, batch_size=32, epochs=10, validation_split=0.1, callbacks=call_backs)

Epoch 1/10
23/23 [==============================] - 201s 9s/step - loss: 8.8002 - acc: 0.9032 - val_loss: 3.7130e-05 - val_acc: 1.0000
Epoch 2/10
23/23 [==============================] - 200s 9s/step - loss: 0.5691 - acc: 0.9643 - val_loss: 1.3374e-08 - val_acc: 1.0000
Epoch 3/10
23/23 [==============================] - 184s 8s/step - loss: 0.2310 - acc: 0.9943 - val_loss: 0.4533 - val_acc: 0.9873
Epoch 4/10
23/23 [==============================] - 185s 8s/step - loss: 0.1903 - acc: 0.9852 - val_loss: 3.1948e-09 - val_acc: 1.0000
Epoch 5/10
23/23 [==============================] - 185s 8s/step - loss: 0.6578 - acc: 0.9792 - val_loss: 4.6255e-07 - val_acc: 1.0000
Epoch 6/10
23/23 [==============================] - 185s 8s/step - loss: 0.0052 - acc: 0.9991 - val_loss: 0.1378 - val_acc: 0.9747
Epoch 7/10
23/23 [==============================] - 185s 8s/step - loss: 0.8338 - acc: 0.9698 - val_loss: 5.6748e-18 - val_acc: 1.0000
Epoch 8/10
23/23 [==============================] - 186s 8s/ste

In [116]:
pred_y = np.round(model.predict(test_x).reshape(len(test_x,)))

In [117]:
(pred_y)

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
       1., 1., 0.], dtype=float32)

In [118]:
accuracy_score(test_y, pred_y)

0.9886363636363636

In [32]:
def pdf_to_img(path_of_pdf, filename):
    
    pdf = pdf2image.convert_from_path(path_of_pdf+'/'+filename)
    i=0
    if 'images' not in os.listdir(path_of_pdf):
        os.makedirs(path_of_pdf +'/images')
    #print(path_of_pdf +'/images')
    for img in pdf:
        img.save(path_of_pdf +'/images/'+filename+str(i)+'.jpg', 'JPEG')
        #print(path_of_pdf +'/images/'+filename+str(i)+'.jpg')
        break
        i = i+1
    
    images = []
    for img in os.listdir(path_of_pdf +'/images'):
        #print(img)
        if img == '.DS_Store':
            pass
        else:
            pic = image.load_img(path_of_pdf +'/images'+'/'+img, target_size=(300, 300))
            images.append(image.img_to_array(pic))
            
    for item in os.listdir(path_of_pdf +'/images'):
        os.remove(path_of_pdf +'/images'+'/'+item)
    
    return np.array(images)/255

In [135]:
new_img = pdf_to_img('/Users/skylark/Desktop', 'resume.pdf')

In [93]:
new_img = image.img_to_array(image.load_img('/Users/skylark/Desktop/1.jpg', target_size=(300,300)))/255

In [129]:
model.save('/Users/skylark/Desktop/model_margin_identification.h5')

In [129]:
cvs = os.listdir('/Users/skylark/Desktop/margin_identification/cv')

In [130]:
cvs[0][:-4]

'.DS_S'

In [136]:
np.round(model.predict(new_img.reshape(1,300,300,3)))

array([[1.]], dtype=float32)

In [208]:
cvs = os.listdir('/Users/skylark/Desktop/margin_identification/cv')
cvs

['.DS_Store',
 'zameermohammad786@gmail.com07062019110017_resume.pdf',
 'sparshupdatedresume.pdf',
 'images',
 'Offer Letter - Mr. Sparsh Srivastava- Associate Data Scientist.pdf',
 'ziyak3438@gmail.com05072018095122_resume.pdf']

In [216]:
margin_dict = {}
for cv in cvs:
    if (cv != ".DS_Store") and (cv!='images'):
        img = pdf_to_img('/Users/skylark/Desktop/margin_identification/cv', cv)
        pred = np.round(model.predict(img)[0])
        if pred == 1:            
             margin_dict[cv[:-4]] = 'Has 1 inch margin'
        else:
            margin_dict[cv[:-4]] = "Doesn't have 1 inch margin" 

In [217]:
margin_dict

{'zameermohammad786@gmail.com07062019110017_resume': "Doesn't have 1 inch margin",
 'sparshupdatedresume': 'Has 1 inch margin',
 'Offer Letter - Mr. Sparsh Srivastava- Associate Data Scientist': 'Has 1 inch margin',
 'ziyak3438@gmail.com05072018095122_resume': "Doesn't have 1 inch margin"}

In [218]:
import glob

In [60]:
pdf = pdf2image.convert_from_path('/Users/skylark/Desktop/resume.pdf')

In [61]:
i=0
for img in pdf:
    img.save('/Users/skylark/Desktop/'+str(i)+'.jpg', 'JPEG')
    i=i+1

In [130]:
from transformers import TFA